In [1]:
import pandas as pd
import numpy as np
import random
import re
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [20]:
animes=pd.read_csv('anime.csv')
sypnosis=pd.read_csv('anime_with_synopsis.csv')
user_ratings=pd.read_csv('animelist.csv')

In [3]:
len(user_ratings['anime_id'].unique())

17562

In [4]:
animes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         17562 non-null  int64 
 1   Name           17562 non-null  object
 2   Score          17562 non-null  object
 3   Genres         17562 non-null  object
 4   English name   17562 non-null  object
 5   Japanese name  17562 non-null  object
 6   Type           17562 non-null  object
 7   Episodes       17562 non-null  object
 8   Aired          17562 non-null  object
 9   Premiered      17562 non-null  object
 10  Producers      17562 non-null  object
 11  Licensors      17562 non-null  object
 12  Studios        17562 non-null  object
 13  Source         17562 non-null  object
 14  Duration       17562 non-null  object
 15  Rating         17562 non-null  object
 16  Ranked         17562 non-null  object
 17  Popularity     17562 non-null  int64 
 18  Members        17562 non-n

In [5]:
sypnosis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16214 entries, 0 to 16213
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MAL_ID     16214 non-null  int64 
 1   Name       16214 non-null  object
 2   Score      16214 non-null  object
 3   Genres     16214 non-null  object
 4   sypnopsis  16206 non-null  object
dtypes: int64(1), object(4)
memory usage: 633.5+ KB


In [78]:
animes=animes.iloc[:,:25].drop(['Japanese name'],axis=1)

In [7]:
animes.head(2)

,MAL_ID,Name,Score,Genres,English name,Type,Episodes,Aired,Premiered,Producers,...,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,...,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",...,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964


In [8]:
sypnosis.head(2)

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."


In [79]:
animes_syp=animes.merge(sypnosis.drop(['Name','Score','Genres'],axis=1),on='MAL_ID',validate='1:1')

In [48]:
animes_syp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16214 entries, 0 to 16213
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         16214 non-null  int64 
 1   Name           16214 non-null  object
 2   Score          16214 non-null  object
 3   Genres         16214 non-null  object
 4   English name   16214 non-null  object
 5   Type           16214 non-null  object
 6   Episodes       16214 non-null  object
 7   Aired          16214 non-null  object
 8   Premiered      16214 non-null  object
 9   Producers      16214 non-null  object
 10  Licensors      16214 non-null  object
 11  Studios        16214 non-null  object
 12  Source         16214 non-null  object
 13  Duration       16214 non-null  object
 14  Rating         16214 non-null  object
 15  Ranked         16214 non-null  object
 16  Popularity     16214 non-null  int64 
 17  Members        16214 non-null  int64 
 18  Favorites      16214 non-n

In [80]:
animes_syp=animes_syp.dropna()

In [12]:
user_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109224747 entries, 0 to 109224746
Data columns (total 5 columns):
 #   Column            Dtype
---  ------            -----
 0   user_id           int64
 1   anime_id          int64
 2   rating            int64
 3   watching_status   int64
 4   watched_episodes  int64
dtypes: int64(5)
memory usage: 4.1 GB


In [81]:
user_ratings.head(2)

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4


In [14]:
user_ratings250 = user_ratings[user_ratings.groupby("user_id")['user_id'].transform('size') > 249]

In [15]:
len(user_ratings250)

89942646

In [16]:
random.seed(42)
users=random.sample(sorted(user_ratings250['user_id'].unique().tolist()),1000)

In [17]:
len(users)

1000

In [50]:
animeids=sorted(animes_syp['MAL_ID'].unique().tolist())

In [51]:
len(animeids)

16206

In [29]:
ratings_matrix=pd.DataFrame(index=animeids)
for i in users:
    animes_by_user=user_ratings250[user_ratings250['user_id']==i][['anime_id','rating']]
    if len(animes_by_user)<250:
        continue
    else:
        watched_animes=[int(animes_by_user.loc[animes_by_user.anime_id==j]['rating']) if j in animes_by_user.anime_id.to_list() else 0 for j in animeids]
        ratings_matrix[i]=watched_animes

In [33]:
ratings_matrix=ratings_matrix[ratings_matrix.sum(axis=1)>0]

In [39]:
ratings_index=ratings_matrix.index.to_list()

In [57]:
ratings_index=[list(animes_syp.loc[animes_syp.MAL_ID==i]['Name'])[0] for i in ratings_index]

In [58]:
ratings_matrix.index=ratings_index

In [4]:
#ratings_matrix.to_csv('ratings_matrix.csv')

In [5]:
dist_eucl = 1 / (1 + squareform(pdist(ratings_matrix.T, metric='euclidean')))

In [6]:
distances = pd.DataFrame(dist_eucl, index=ratings_matrix.T.index, columns=ratings_matrix.T.index)

In [2]:
ratings_matrix=pd.read_csv('ratings_matrix.csv',index_col=0)

In [3]:
def colab_recommendations_euc(d,ratings_matrix=ratings_matrix.T):
    matrix=ratings_matrix.loc[:,list(d.keys())]
    matrix=matrix[matrix.isin(range(1,11))].dropna()
    matrix.loc['New_User'] = list(d.values())
    dist_eucl = 1 / (1 + squareform(pdist(matrix, metric='euclidean')))
    distances = pd.DataFrame(dist_eucl, index=matrix.index, columns=matrix.index)
    similarities = distances['New_User'].sort_values(ascending=False)[1:]
    similarities=similarities.to_frame()
    animes_to_recommend=ratings_matrix.drop(list(d.keys()),axis=1)
    animes_to_recommend=animes_to_recommend.loc[list(distances.columns)[:-1]]
    animes_to_recommend=animes_to_recommend.T
    for i in animes_to_recommend.columns:
        animes_to_recommend.loc[:,i]=animes_to_recommend.loc[:,i]*float(similarities.loc[i])
    animes_to_recommend['Total'] = animes_to_recommend.sum(axis=1)
    animes_to_recommend=animes_to_recommend.sort_values('Total', ascending=False)['Total'].to_frame()
    animes_to_recommend.loc[:,'Total']=round(animes_to_recommend.loc[:,'Total']*10/animes_to_recommend['Total'].max(),2)
    return animes_to_recommend.head(10)

In [4]:
def colab_recommendations_cos(d,ratings_matrix=ratings_matrix.T):
    matrix=ratings_matrix.loc[:,list(d.keys())]
    matrix=matrix[matrix.isin(range(1,11))].dropna()
    matrix.loc['New_User'] = list(d.values())
    dist_eucl = 1 / (1 + squareform(pdist(matrix, metric='cosine')))
    distances = pd.DataFrame(dist_eucl, index=matrix.index, columns=matrix.index)
    similarities = distances['New_User'].sort_values(ascending=False)[1:]
    similarities=similarities.to_frame()
    animes_to_recommend=ratings_matrix.drop(list(d.keys()),axis=1)
    animes_to_recommend=animes_to_recommend.loc[list(distances.columns)[:-1]]
    animes_to_recommend=animes_to_recommend.T
    for i in animes_to_recommend.columns:
        animes_to_recommend.loc[:,i]=animes_to_recommend.loc[:,i]*float(similarities.loc[i])
    animes_to_recommend['Total'] = animes_to_recommend.sum(axis=1)
    animes_to_recommend=animes_to_recommend.sort_values('Total', ascending=False)['Total'].to_frame()
    animes_to_recommend.loc[:,'Total']=round(animes_to_recommend.loc[:,'Total']*10/animes_to_recommend['Total'].max(),2)
    return animes_to_recommend.head(10)

In [5]:
def colab_recommendation_combined(d):
    combined_df = colab_recommendations_euc(d).merge(colab_recommendations_cos(d), left_index=True, right_index=True)
    combined_df['Total'] = combined_df.sum(axis=1)/2
    combined_df=combined_df.sort_values('Total', ascending=False)['Total'].to_frame()
    return combined_df

In [6]:
Ernesto = {'Death Note': 8,
           'Shingeki no Kyojin': 8,
           'Fullmetal Alchemist: Brotherhood': 9,
           'Sword Art Online': 7,
           'One Punch Man': 8,
           'Boku no Hero Academia': 7,
           'Tokyo Ghoul': 6,
           'Hunter x Hunter (2011)': 9,
           'Steins;Gate': 10,
           'Kimetsu no Yaiba': 9
            }

In [8]:
colab_recommendation_combined(Ernesto)

,Total
Re:Zero kara Hajimeru Isekai Seikatsu,9.990
Shingeki no Kyojin Season 2,9.935
Kimi no Na wa.,9.870
Boku no Hero Academia 2nd Season,9.730
Kiseijuu: Sei no Kakuritsu,9.485
Code Geass: Hangyaku no Lelouch,9.450
No Game No Life,9.355
Code Geass: Hangyaku no Lelouch R2,9.295
Boku dake ga Inai Machi,9.265
Koe no Katachi,9.195
